In [2]:
import numpy as np
import cv2 as cv
from pathlib import Path
import time

path = Path(".")
path = path.glob("airplanes/*.jpg")

count = 0
totalTime = 0

src = cv.cuda_GpuMat()

for imagepath in path:

    src_host = cv.imread(str(imagepath), cv.IMREAD_GRAYSCALE)
    src.upload(src_host)

    start_t = time.time()
    
    gaussian_filter = cv.cuda.createGaussianFilter(src.type(), src.type(), (3, 3), 0, cv.BORDER_DEFAULT)
    gpu_blur = gaussian_filter.apply(src)

    gradX_filter = cv.cuda.createSobelFilter(gpu_blur.type(), gpu_blur.depth(), 0, 1, 3)
    grad_x = gradX_filter.apply(gpu_blur)
    gradY_filter = cv.cuda.createSobelFilter(gpu_blur.type(), gpu_blur.depth(), 1, 0, 3)
    grad_y = gradY_filter.apply(gpu_blur)

    abs_grad_x = cv.cuda.abs(grad_x)
    abs_grad_y = cv.cuda.abs(grad_y)

    gpu_sobel = cv.cuda.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    
    totalTime = totalTime + (time.time() - start_t)

    # gpu_sobel = cv.cuda.add(grad_x, grad_y)

    result_host = gpu_sobel.download()
    
    count += 1;
#     cv.imwrite("output/OP" + str(count) + ".jpg", result_host)

print("ImageCount =", count, "\nTimeRequired =", totalTime)


ImageCount = 800 
TimeRequired = 1.6938116550445557
